<a href="https://colab.research.google.com/github/nbeaudoin/LangChain-Experimentation-Zone/blob/main/LangChain_Basics_Tutorial_2_Tools_and_Chains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [3]:
!pip -q install openai langchain huggingface_hub
!pip install python-dotenv

# Mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

import os
from dotenv import load_dotenv

# Load variables from .env file
load_dotenv('/content/drive/MyDrive/secrets.json')

# Use variables
openai_api = os.getenv('OPENAI_API_KEY')
huggingface_api = os.getenv('HUGGINGFACE_API_KEY')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.9 MB/s eta 0:00:00
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Basic LLMChain - Fact Extraction

In [4]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI

In [5]:
llm = OpenAI(model_name='text-davinci-003',
             temperature=0,
             max_tokens=256)

### Load article on SpongeBob from CNN

In [19]:
article = """
Wall Street stocks popped at the open on Thursday, as investors weighed hotter-than-expected retail sales and wholesale price inflation data for a steer on what path the Federal Reserve will take on rates policy.

The S&P 500 (^GSPC) gained 0.5%, while the Dow Jones Industrial Average (^DJI) ticked up 0.4%, or about 150 points. The Nasdaq Composite (^IXIC) added around 0.6%, as British chipmaker Arm was set to make its stock-market debut.

Despite much anticipation, markets were little moved Wednesday by an uptick in the August consumer inflation report. That wouldn't be enough to prompt the Fed to change course, economists said.

Thursday's reading on retail sales for the same month came in stronger than for July, underscoring how the US consumer has remained resilient despite rising interest rates. Sales grew 0.6% on the month, compared with 0.1% expected, picking up steam as people shelled out more for gas.

The producer price index increased 0.7% in August, up from 0.4% the previous month, data out Thursday showed — a sign that inflation remains stubbornly persistent despite the Fed's efforts to cool pressures. But at the same time, "core" wholesale inflation rose 2.2%, down from July's print of 2.4%."

Eyes will be on Arm's debut on the Nasdaq on Thursday, after the Softbank-backed chip designer priced its blockbuster IPO at the top of the range at $51 per share, for a $54.5 billion valuation.

Also in focus was the ongoing rally in oil prices, given their significant effect on inflation and on stocks. WTI crude (CL=F) and Brent (BZ=F) futures were trading near 10-month highs on Thursday.

Elsewhere, the European Central Bank hiked its interest rates for the 10th time in a row, to 4% — the highest level since the euro was launched in 1999. ECB policymakers also upped their inflation forecast, which they expect to remain too high for too long.
"""

In [31]:
fact_extraction_prompt = PromptTemplate(
    input_variables=["text_input"],
    template="Extract the key facts out of this text into 10 numbered sentences. Don't include opinions. Give each fact a number. :\n\n {text_input}"
)


In [32]:

fact_extraction_chain = LLMChain(llm=llm,
                                 prompt=fact_extraction_prompt)

facts = fact_extraction_chain.run(article)

print(facts)


1. Stocks popped at the open on Thursday as investors weighed hotter-than-expected retail sales and wholesale price inflation data. 
2. The S&P 500 gained 0.5%, while the Dow Jones Industrial Average ticked up 0.4%, or about 150 points. 
3. The Nasdaq Composite added around 0.6%, as British chipmaker Arm was set to make its stock-market debut.
4. Retail sales grew 0.6% on the month in August, compared with 0.1% expected.
5. The producer price index increased 0.7% in August, up from 0.4% the previous month.
6. Arm priced its IPO at the top of the range at $51 per share, for a $54.5 billion valuation.
7. WTI crude and Brent futures were trading near 10-month highs on Thursday.
8. The European Central Bank hiked its interest rates for the 10th time in a row, to 4%.
9. ECB policymakers also upped their inflation forecast, which they expect to remain too high for too long.
10. Oil prices have a significant effect on inflation and on stocks.


## Rewrite as a summary from the facts


In [33]:
investor_update_prompt = PromptTemplate(
    input_variables=["facts"],
    template="You are a Goldman Sachs analyst. Take the following list of facts and use them to write a short paragraph for investors, Don't leave out key info: \n\n {facts}"
)

In [34]:
investor_update_chain = LLMChain(llm=llm,
                                 prompt=investor_update_prompt)

investor_update = investor_update_chain.run(facts)

print(investor_update)
len(investor_update)



Investors were optimistic on Thursday as stocks opened higher, buoyed by better-than-expected retail sales and wholesale price inflation data. The S&P 500 gained 0.5%, while the Dow Jones Industrial Average ticked up 0.4%, or about 150 points. The Nasdaq Composite added around 0.6%, as British chipmaker Arm was set to make its stock-market debut. Retail sales grew 0.6% on the month in August, compared with 0.1% expected, and the producer price index increased 0.7% in August, up from 0.4% the previous month. Arm priced its IPO at the top of the range at $51 per share, for a $54.5 billion valuation. WTI crude and Brent futures were trading near 10-month highs on Thursday, and the European Central Bank hiked its interest rates for the 10th time in a row, to 4%. ECB policymakers also upped their inflation forecast, which they expect to remain too high for too long. Oil prices have a significant effect on inflation and on stocks, and investors are keeping a close eye on these developments

1001

In [35]:
triples_chain = PromptTemplate(
    input_variables=["facts"],
    template="Take the following list of facts and turn them into triples for a knowledge graph: \n\n {facts}"
)

In [36]:
triples_chain = LLMChain(llm=llm,
                         prompt=triples_chain)

triples = triples_chain.run(facts)

print(triples)
len(triples)




(1) (Subject: Stocks, Predicate: popped, Object: at the open on Thursday)
(2) (Subject: S&P 500, Predicate: gained, Object: 0.5%)
(3) (Subject: Dow Jones Industrial Average, Predicate: ticked up, Object: 0.4%)
(4) (Subject: Retail sales, Predicate: grew, Object: 0.6% on the month in August)
(5) (Subject: Producer price index, Predicate: increased, Object: 0.7% in August)
(6) (Subject: Arm, Predicate: priced, Object: IPO at the top of the range at $51 per share)
(7) (Subject: WTI crude, Predicate: trading, Object: near 10-month highs on Thursday)
(8) (Subject: European Central Bank, Predicate: hiked, Object: interest rates for the 10th time in a row to 4%)
(9) (Subject: ECB policymakers, Predicate: upped, Object: inflation forecast)
(10) (Subject: Oil prices, Predicate: have, Object: significant effect on inflation and on stocks


841

## Chaining these together


In [44]:
from langchain.chains import SimpleSequentialChain, SequentialChain

full_chain = SimpleSequentialChain(chains=[fact_extraction_chain, investor_update_chain], verbose=True)

full_chain.run(article)





> Entering new SimpleSequentialChain chain...

1. Stocks popped at the open on Thursday as investors weighed hotter-than-expected retail sales and wholesale price inflation data. 
2. The S&P 500 gained 0.5%, while the Dow Jones Industrial Average ticked up 0.4%, or about 150 points. 
3. The Nasdaq Composite added around 0.6%, as British chipmaker Arm was set to make its stock-market debut.
4. Retail sales for August grew 0.6% on the month, compared with 0.1% expected.
5. The producer price index increased 0.7% in August, up from 0.4% the previous month.
6. Arm priced its IPO at the top of the range at $51 per share, for a $54.5 billion valuation.
7. WTI crude and Brent futures were trading near 10-month highs on Thursday.
8. The European Central Bank hiked its interest rates for the 10th time in a row, to 4%.
9. ECB policymakers also upped their inflation forecast, which they expect to remain too high for too long.
10. Oil prices have a significant effect on inflation and on stocks.


'\n\nThursday saw stocks open higher as investors reacted to better-than-expected retail sales and wholesale price inflation data. The S&P 500, Dow Jones Industrial Average, and Nasdaq Composite all gained, with the Nasdaq boosted by the stock-market debut of British chipmaker Arm. Retail sales for August grew 0.6%, while the producer price index increased 0.7%. Oil prices, which have a significant effect on inflation and stocks, were trading near 10-month highs, and the European Central Bank hiked its interest rates for the 10th time in a row, while also upping its inflation forecast.'

# API Chains - OpenMeteo - Weather Information


In [49]:
from langchain import OpenAI
from langchain.chains.api.prompt import API_RESPONSE_PROMPT

from langchain.chains import APIChain
from langchain.prompts.prompt import PromptTemplate



In [50]:
llm = OpenAI(temperature=0,
             max_tokens=100)


In [52]:
from langchain import chains
from langchain.chains.api import open_meteo_docs

chain_new = APIChain.from_llm_and_api_docs(llm,
                                           open_meteo_docs.OPEN_METEO_DOCS,
                                           verbose=True)



In [53]:
chain_new.run('What is the temperature right now in Pasadena, California in degrees Fahrenheit?')




> Entering new APIChain chain...
https://api.open-meteo.com/v1/forecast?latitude=34.147785&longitude=-118.144516&hourly=temperature_2m&temperature_unit=fahrenheit&current_weather=true
{"latitude":34.156918,"longitude":-118.16109,"generationtime_ms":0.2510547637939453,"utc_offset_seconds":0,"timezone":"GMT","timezone_abbreviation":"GMT","elevation":267.0,"current_weather":{"temperature":64.1,"windspeed":1.5,"winddirection":284,"weathercode":3,"is_day":1,"time":"2023-09-14T14:00"},"hourly_units":{"time":"iso8601","temperature_2m":"°F"},"hourly":{"time":["2023-09-14T00:00","2023-09-14T01:00","2023-09-14T02:00","2023-09-14T03:00","2023-09-14T04:00","2023-09-14T05:00","2023-09-14T06:00","2023-09-14T07:00","2023-09-14T08:00","2023-09-14T09:00","2023-09-14T10:00","2023-09-14T11:00","2023-09-14T12:00","2023-09-14T13:00","2023-09-14T14:00","2023-09-14T15:00","2023-09-14T16:00","2023-09-14T17:00","2023-09-14T18:00","2023-09-14T19:00","2023-09-14T20:00","2023-09-14T21:00","2023-09-14T22:00","20

' The temperature right now in Pasadena, California is 64.1°F.'

In [54]:
chain_new.run("Is it foggy in Pasadena, California?")



> Entering new APIChain chain...
https://api.open-meteo.com/v1/forecast?latitude=34.147785&longitude=-118.144516&hourly=weathercode&timezone=auto&current_weather=true
{"latitude":34.156918,"longitude":-118.16109,"generationtime_ms":0.6619691848754883,"utc_offset_seconds":-25200,"timezone":"America/Los_Angeles","timezone_abbreviation":"PDT","elevation":267.0,"current_weather":{"temperature":17.9,"windspeed":1.5,"winddirection":284,"weathercode":3,"is_day":1,"time":"2023-09-14T07:00"},"hourly_units":{"time":"iso8601","weathercode":"wmo code"},"hourly":{"time":["2023-09-14T00:00","2023-09-14T01:00","2023-09-14T02:00","2023-09-14T03:00","2023-09-14T04:00","2023-09-14T05:00","2023-09-14T06:00","2023-09-14T07:00","2023-09-14T08:00","2023-09-14T09:00","2023-09-14T10:00","2023-09-14T11:00","2023-09-14T12:00","2023-09-14T13:00","2023-09-14T14:00","2023-09-14T15:00","2023-09-14T16:00","2023-09-14T17:00","2023-09-14T18:00","2023-09-14T19:00","2023-09-14T20:00","2023-09-14T21:00","2023-09-14T22:

' The current weather conditions in Pasadena, California are 17.9°C with a windspeed of 1.5 km/h and a weathercode of 3, which indicates fog.'